<h1 style='text-align: center;'> Final Project 4 : Clustering with K-Means Clustering </h1>

## Created by: Kelompok 9
### Class: PYTN-KS18

- 👤 **Member 1:** Vincent Tanaka - PYTN-KS18-013
- 👤 **Member 2:** Audris Vondrea Wirduno - PYTN-KS18-02


## a. Introduction

### Brief

<div align="justify">

*Project* ini akan membahas mengenai penggunaan *clustering* yang merupakan teknik *unsupervised machine learning* pada *dataset* pengguna kartu kredit yang terdiri dari ***9000*** pengguna. Hasil akhir yang diharapkan adalah untuk mendapatkan strategi marketing yang efektif dari hasil *clustering* yang dilakukan.

</div>

### a.1 Background

<div align="justify">

Pada *notebook* ini studi kasus yang dibahas berupa prilaku **9000** pengguna kartu kredit aktif dalam waktu 6 Bulan. Dengan jumlah variabel sebanyak **18**. Hasil yang diharapkan dari studi kasus ini dalah untuk membuat sebuah strategi marketing dari *clustering* yang akan dilakukan setelahnya.  
Data didapatkan dari kaggle dengan link : https://www.kaggle.com/datasets/arjunbhasin2013/ccdata

</div>

### a.2 About Dataset

Berikut adalah deskripsi **18** variabel dari *dataset* yang akan digunakan:  
1. **CUSTID** - Identification of Credit Card holder (Categorical)
2. **BALANCE** - Balance amount left in their account to make purchases
3. **BALANCEFREQUENCY** - How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
4. **PURCHASES** - Amount of purchases made from account
5. **ONEOFFPURCHASES** - Maximum purchase amount done in one-go
6. **INSTALLMENTSPURCHASES** - Amount of purchase done in installment
7. **CASHADVANCE** - Cash in advance given by the user
8. **PURCHASESFREQUENCY** - How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
9. **ONEOFFPURCHASESFREQUENCY** - How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
10. **PURCHASESINSTALLMENTSFREQUENCY** - How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
11. **CASHADVANCEFREQUENCY** - How frequently the cash in advance being paid
12. **CASHADVANCETRX** - Number of Transactions made with "Cash in Advance"
13. **PURCHASESTRX** - Number of purchase transactions made
14. **CREDITLIMIT** - Limit of Credit Card for user
15. **PAYMENTS** - Amount of Payment done by user
16. **MINIMUM_PAYMENTS** - Minimum amount of payments made by user
17. **PRCFULLPAYMENT** - Percent of full payment paid by user
18. **TENURE** - Tenure of credit card service for user

### a.3 Project Objectives

Adapun *Objective* dari pengerjaan *final project* ini yaitu :  
1. Melakukan eksplorasi data mengenai hubungan antar variabel dari data.
2. Melakukan *preprocessing data* sebelum pemodelan data.
3. Melakukan pemodelan data *clustering* yang sudah ditentukan
4. Mengambil kesimpulan dari apa yang sudah dilakukan.

## b. Import Libraries

In [ ]:
# import the needed libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import time
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from kneed import KneeLocator
from sklearn.metrics import silhouette_score

## c. *Dataset Loading*

In [ ]:
# Import the dataset given, name it as df
df = pd.read_csv('cc-dataset.zip',compression='zip')

In [ ]:
# Show top 5 row of df
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

<div align="justify">

Disini bisa disimpulkan bahwa *dataset* yang akan digunakan memiliki data sebanyak **8950** baris dengan jumlah variabel sebanyak **18** kolom.

</div>

## d. Data Cleaning

### d.1 Duplicate Data Handling

In [ ]:
# Check if there's a duplicated data
df.duplicated().sum()

Dari hasil pengecekan tidak ada data duplikat pada *dataset* ini, proses dilanjut dengan meng-*handle missing values*.

### d.2 Missing Data Handling

In [ ]:
df.isna().sum()

Terdapat **2** kolom yang memiliki *missing values*, yaitu **CREDIT_LIMIT(1 missing value)** dan **MINIMUM_PAYMENTS(313 missing values)**. Maka dari itu data tersebut akan dihandle menggunakan inputasi. Pertama-tama bisa dicek terlebih dahulu distribusi data pada kedua kolom tersebut.

In [ ]:
# List of columns with missing values to plot
columns_to_plot = ['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Create a figure and set its size
plt.figure(figsize=(12, 6))

# Loop through the missing value columns list and create a subplot for each
for i, column in enumerate(columns_to_plot, 1):
    plt.subplot(1, 2, i)  # 1 row, 2 columns, subplot index
    sns.histplot(df[column].dropna(), bins=30, kde=False, edgecolor='black')
    plt.title(f'Data Distribution of {column}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


Dari hasil ini bisa disimpulkan bahwa distribusi data kedua kolom yang memiliki *missing values* cenderung ke *right-skewed* dimana nilai *mean* < nilai *median*. Dengan kondisi tersebut data yang kosong akan diisi dengan nilai median dikarenakan distribusi data tidak normal.

In [ ]:
# define a list of columns that need to be filled with median number
columns_to_fill = ['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']

for columns in columns_to_fill:
    median_value = df[columns].median()
    df[columns].fillna(median_value, inplace=True)

# Verify if NaN values have been filled
df.isna().sum()

Disini data sudah tidak memiliki nilai kosong, proses bisa dilanjut dengan membuang kolom yang tidak diperlukan.

### d.3 Dropping un-needed Columns

In [ ]:
df.info()

In [ ]:
df.head(10)

Dari hasil tampilan *dataframe*, bisa disimpulkan bahwa kolom **CUST_ID** merupakan sebuah *primary key* dari *dataset*. Dikarenakan nilai didalamnya merupakan *unique value* yang berjumlah sebanyak **8950** *unique values*, maka kelompok 9 akan menetapkan bahwa kolom ini akan dibuang.

In [ ]:
df.drop(columns='CUST_ID', inplace=True)
df

Sekarang, data uang akan digunakan hanya terdiri dari **17** kolom saja. Proses bisa dilanjutkan dengan melakukan *EDA*.

## e. Explanatory Data Analysis

### e.1 Measure of Central Tendency

In [ ]:
# Calculate the Central Tendency of each numerical columns
central_tendency = df.describe().T
central_tendency.reset_index().rename(
    columns={'index': 'Attributes'})

*DataFrame* tersebut menghitung beberapa *measure of central tendency* untuk setiap kolom yang memiliki tipe data numerik. *Measure of central tendency* yang dihitung melibatkan statistik deskriptif dasar, termasuk *mean* (rata-rata), *std* (standar deviasi), *min* (nilai minimum), 25% (kuartil pertama), 50% (median atau kuartil kedua), 75% (kuartil ketiga), dan *max* (nilai maksimum). *DataFrame* ini memberikan gambaran singkat tentang sebaran nilai di setiap kolom dan membantu dalam memahami *central tendency* dari dataset tersebut.

### e.2 Measure of Central Tendency

In [ ]:
# Create a dataframe that only contains numeric columns
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
numeric_df = df[numerical_columns]

In [ ]:
# Compute some measure of variability
range_values = numeric_df.max() - numeric_df.min()
variance_values = numeric_df.var()
std_deviation_values = numeric_df.std()
iqr_values = numeric_df.quantile(0.75) - numeric_df.quantile(0.25)
cv_values = std_deviation_values / numeric_df.mean()

In [ ]:
# Combine the results into one dataframe
variability_measures = pd.DataFrame({
    'Range': range_values,
    'Variance': variance_values,
    'Std Deviation': std_deviation_values,
    'IQR': iqr_values,
    'CV': cv_values
})
variability_measures.head()

Dalam hasil diatas, telah dihitung beberapa ukuran variabilitas (*measure of variability*) untuk kolom-kolom numerik pada dataset pengguna kartu kredit dari *Kaggle*. Ini mencakup *Range* (Rentang), *Variance* (Varians), *Std Deviation* (Deviasi Standar), IQR (*Interquartile Range*), dan CV (*Coefficient of Variation*). Hasil ini memberikan gambaran tentang sebaran dan variasi data keuangan, seperti rentang batas kredit, variasi nilai transaksi, dan konsistensi tagihan. DataFrame yang dihasilkan menyajikan nilai-nilai tersebut untuk setiap kolom numerik dalam dataset, memberikan wawasan komprehensif terhadap karakteristik variabilitas data keuangan.

### e.3 Simple Data Plotting

In [ ]:
# List of your DataFrame's columns
columns = df.columns

# Determine the number of rows/columns for the subplot grid
n_rows = (len(columns) + 1) // 2
n_cols = 2

# Create a figure and a set of subplots
plt.figure(figsize=(15, 5 * n_rows))

for i, column in enumerate(columns):
    plt.subplot(n_rows, n_cols, i + 1)
    # Check if the data is numerical or categorical
    if df[column].dtype in ['int64', 'float64']:
        sns.histplot(df[column], kde=True, bins=30)
    else:
        sns.countplot(x=column, data=df)
    plt.title(column)
    plt.tight_layout()

plt.show()

Hasil *plot* diatas merupakan distribusi data yang ada pada tiap kolom *dataset*. Ada beberapa hal yang bisa disimpulkan dari hasil ini yaitu:
- Banyak kolom yang memiliki grafik *right-skewed* dimana data-data tersebut memiliki nilai mean < dari nilai mediannya.
- Pada kolom **'BALANCE'** dan **'BALANCE_FREQUENCY'** bisa disimpulkan bahwa para pengguna kartu kredit rajin dalam membayar kredit mereka bisa dilihar dengan kolom **'BALANCE'** yang sering muncul adalah 0. Diikuti dengan kolom **'BALANCE_FREQUENCY'** yang memiliki mayoritas nilai 1.0 yang membuktikan berapa seringnya kartu kredit tersebut digunakan.
- Pada kolom **'TENURE'** bisa disimpulkan bahwa pengguna kartu kredit, lebih memilih pembayaran selama 12 bulan dalam kreditnya meskipun biasanya pembayaran dengan bulan yang lebih lama itu jauh lebih mahal di suku bunganya.

In [ ]:
# Create a scatter plot of CREDIT_LIMIT vs BALANCE with a hue based on TENURE
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='CREDIT_LIMIT', y='BALANCE', hue='TENURE', palette='viridis', s=100)

# Add plot title and labels
plt.title('Scatter Plot of Credit Limit vs Balance by Tenure (Grouped)')
plt.xlabel('Credit Limit')
plt.ylabel('Balance')
plt.legend(title='Tenure')

# Show the plot
plt.show()

Bisa dilihat dari pesebaran data berdasarkan kolom **'BALANCE'** dan **'CREDIT_LIMIT'** pengguna lebih banyak menggunakan TENURE selama 12 bulan. Dibuktikan dari titik bewarna kuning pada plot ini.

## f. Data Preprocessing

### f.1 Correlation Check

In [ ]:
corr = df.corr()
# Set up the matplotlib figure
plt.figure(figsize=(12, 10))
# Choose a diverging color palette
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Generate a heatmap
sns.heatmap(corr, annot=True, cmap=cmap, linewidths=.5)
# Show the plot
plt.show()

Berikut adalah hasil *plot* korelasi yang dilakukan pada *DataFrame* menggunakan *Heatmap* dimana warna korelasi yang mengarah ke angka (1.0) atau merah menunjukkan berapa positifnya korelasi antar kolom yang ada *DataFrame*. Begitu sebaliknya dengan yang mengarak ke angka (0.) atay biru yang menunjukkan bahwa kolom tersebut memiliki korelasi negatif atau tidak berhubungan satu sama lain. Disini ada 1 kolom yang memiliki korelasi yang sangat kuat dengan kolom lainnya dengan nilai korelasi sebanyak **0.92** yaitu **'ONEOFF_PURCHASES'**.

### f.2 *Standardization*

<div align='justify'>
Proses selanjutnya adalah scaling dataset. Disini dataset akan diubah menggunakan transformasi linear untuk menghasilkan cluster yang lebih bagus kualitasnya dengan mengelola variabilitas dataset.Standarisasi dataset akan diharapkan untuk meningkatkan peforma clustering nantinya.
</div>

In [ ]:
# Scale the entire column
X = pd.DataFrame(StandardScaler().fit_transform(df), columns=df.columns)

In [ ]:
X

### f.3 *Inferential Statistic Test with a Hopkins Test*

Proses selanjutnya adalah untuk melakukan statistik inferensial menggunakan *Test Hopkins* untuk menghitung *clustering tendency*/kecenderungan pengelompokan data sekarang.
Berikut adalah penjelasan Hipotesis Hopkins Test yang akan dilakukan :
- H0: Dataset mungkin diclusterkan (contains meaningful clusters).
- H1: Dataset tidak mungkin diclusterkan (no meaningful clusters).

In [ ]:
def hopkins_statistic(X):
    n, d = X.shape
    m = int(0.1 * n)  # considering 10% of the total data
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)

    rand_X = np.random.uniform(X.min(), X.max(), (m, d))
    
    uj = np.array([nbrs.kneighbors([rand_X[i]], return_distance=True)[0][0][0] for i in range(m)])
    wj = np.array([nbrs.kneighbors([X.iloc[random.choice(range(n))]], return_distance=True)[0][0][0] for i in range(m)])

    H = np.sum(uj) / (np.sum(uj) + np.sum(wj))
    return H

# Assuming 'X' is your standardized DataFrame
hopkins_score = hopkins_statistic(X)
print("Hopkins statistic:", hopkins_score)

Hasil pengecekan statistik hopkins diatas, bisa disimpulkan bahwa H0 diterima dimana dataset ini memiliki cluster yang berguna. Proses bisa dilanjut dengan implementasi PCA untuk mereduksi dimensi data.

### f.4 *Dimensionality Reduction with PCA*

*PCA (principal component analysis)* merupakan metode yang biasa digunakan di *unsupervised machine learning* untuk meringkas tabel data multivariat dalam skalar besar sehingga bisa dijadikan variabel yang lebih kecil.

In [ ]:
# Instantiate PCA
pca = PCA(n_components=2)

# Apply PCA to the data
X_pca = pca.fit_transform(X)

# Convert to DataFrame for easier analysis
X_pca_df = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])

## g. *Define the Model*

<div align='justify'>

Disini metode *clustering* yang akan dilakukan adalah *K-Means Clustering*. *K-Means Clustering* adalah metode *unsupervised machine mearning* yang biasa digunakan untuk *clustering*. Biasanay prosedurnya berupa klasifikasi kumpulan data tertentu kedalam sejumlah kluster yang ditentukan dengan huruf 'K' yang telah ditetapkan sebelumnya.  
Sebelum dilakukan clustering, ada tahap yang bisa dilakukan sebelumnya yaitu untuk menentukan nilai K yang paling optimal.

</div>

In [ ]:
# Range of K values to try
K_range = range(1, 10)

# Lists to store results
inertia = []
times = []

# Compute KMeans and record inertia and time for each k
for k in K_range:
    start_time = time.time()
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X)
    end_time = time.time()
    
    # Append inertia and time
    inertia.append(kmeans.inertia_)
    times.append(end_time - start_time)

# Determine the elbow point using the kneed library
kneedle = KneeLocator(K_range, inertia, curve='convex', direction='decreasing')
elbow_k = kneedle.elbow

# Plotting the results
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting the inertia (distortion scores)
ax1.plot(K_range, inertia, 'bo-', label='Distortion Score')
ax1.set_xlabel('Number of Clusters, K')
ax1.set_ylabel('Distortion Score', color='b')
ax1.tick_params('y', colors='b')
ax1.set_title('Distortion Score and Computation Time for Different K Values')

# Adding the Elbow annotation
ax1.axvline(x=elbow_k, color='black', linestyle='--', label=f'Elbow at K={elbow_k}')

# Creating a twin y-axis to plot the computation time
ax2 = ax1.twinx()
ax2.plot(K_range, times, 'rv--', label='Fit Time (seconds)')
ax2.set_ylabel('Fit Time (seconds)', color='r')
ax2.tick_params('y', colors='r')

# Adding legends
fig.legend(loc='upper right', bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)

# Show the plot
plt.show()


Contoh diatas merupakan implemetasi *elbow method* untuk menentukan nilai K terbaik untuk *dataframe* ini. Didapatkan nilai **K** paling optimal adalah **4** cluster.

In [ ]:
# Define the model with the optimal value on K
final_kmeans = KMeans(n_clusters=elbow_k, n_init=10, random_state=42)

## h. Train the Model

Disini proses dilanjut dengan melatih model menggunakan variabel yang sudah ditetapkan sebelumnya (X).

In [ ]:
# Fit the model to your data
final_kmeans.predict(X)

## i. *Evaluate the Model*

In [ ]:
silhouette_avg = silhouette_score(X, clusters)
print("The average silhouette_score is :", silhouette_avg)